In [1]:
import csv
def open_file(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        header = next(reader)
        tw_dict = {}
        for i in reader:
            if(i[1] in tw_dict):
                tw_dict[i[1]].append(i[2])
            else:
                tw_dict[i[1]] = []
                tw_dict[i[1]].append(i[2])
        return tw_dict

In [2]:
import re

def find_user(text):
    pattern = re.compile(r'@USER_\w{8}')
    result = pattern.findall(text)
    pos = 0
    for pos in range(len(result)):
        result[pos] = result[pos][1:]
    return result

def relation_dict(tw_dict):
    relation_dict = {}
    l = []
    for user in tw_dict:
        txs = tw_dict[user]
        one_rl = {}
        for tx in txs:
            rl_users = find_user(tx)
            for one_user in rl_users:
                if(one_user in one_rl):
                    one_rl[one_user] += 1
                else:
                    one_rl[one_user] = 1
                    
        if(len(one_rl) > 0):
            relation_dict[user] = one_rl
#         else:
#             print(user, "no friends!!")

    return relation_dict

In [3]:
def direct_link(rl_dict, user):
    dl_dict = {}
    l_users = rl_dict[user]
    for l_user in l_users:
        if(l_user in rl_dict and l_user != user):
#             print("There is a linked user is", l_user, "with mentioned time", l_users[l_user])
            dl_dict[l_user] = l_users[l_user]
            if(user in rl_dict[l_user]):
#                 print("Linked back user is", l_user, "with mentioned time", rl_dict[l_user][user])
                dl_dict[l_user] += rl_dict[l_user][user]
    return dl_dict

In [4]:
def is_celebrity(user, rl_dict, threshhold):
    if(user not in rl_dict):
        count = 0
        for i_user in rl_dict:
            if(user in rl_dict[i_user]):
                count += 1
        if(count > threshhold):
#             print("celebrity is", user)
            return True
    return False
    
    
def indirect_link(rl_dict, user, threshhold):
    idl_dict = {}
    l_users = rl_dict[user]
    for l_user in l_users:
        if(l_user != user):
            for o_user in rl_dict:
                if(o_user != user):
                    ol_users = rl_dict[o_user]
                    if(l_user in ol_users and not is_celebrity(l_user, rl_dict, threshhold)):
#                         print("User", user, "and", o_user, "both mentioned", l_user, l_users[l_user] + ol_users[l_user], "times")
                        idl_dict[o_user] = l_users[l_user] + ol_users[l_user]
    return idl_dict

In [5]:
def generate_network(rl_dict, celebrity_threshhold):
    network = {}
    for user in rl_dict:
        dl = direct_link(rl_dict, user)
        indl = indirect_link(rl_dict, user, celebrity_threshhold)
        link = {}
        for l in dl:
            link[l] = dl[l]
#             if(l in indl):
#                 link[l] += indl[l]
        for l in indl:
            if(l not in link):
                link[l] = indl[l]
        if(len(link) > 0):
            network[user] = link
    return network

In [8]:
import networkx as nx
from node2vec import Node2Vec
def node2vec_300(network, tw_dict):
    g = nx.Graph()
    for user in network:
        for l_user in network[user]:
            g.add_edge(user, l_user, weight = network[user][l_user])
    for o_user in tw_dict:
        if(o_user not in network):
            g.add_node(o_user)
    node2vec = Node2Vec(g, dimensions = 300, walk_length = 200, num_walks = 20, p = 0.25, q = 0.25, workers = 8)
    model = node2vec.fit()
#     l = model.wv.vectors
#     node = model.wv.key_to_index 
    
    return model

In [7]:
import csv
import numpy as np
def get_vector(user, l, node):
    if(user in node):
        return l[node[user]]
    else:
        o = []
        for i in range(300):
            o.append(0)
        return np.array(o)

def generate_file(filename, output_name, l, node):
    out_f = open("%s_node2vec300.csv"%output_name, 'w', newline='')
    writer = csv.writer(out_f)
    
    with open(filename) as f:
        reader = csv.reader(f)
        header = next(reader)
        writer.writerow(header)
        alin = []
        for i in reader:
            if(i[1] not in alin):
                alin.append(i[1])
                writer.writerow((i[0], i[1], str(get_vector(i[1], l, node))))
        print(len(alin))
    out_f.close()

# generate_file('D:/comp90049/project3/data/train_full.csv', 'train', l, node)

In [9]:
def work_flow(filename, output_name):
    tw_dict = open_file(filename)
    rl_dict = relation_dict(tw_dict)
    network = generate_network(rl_dict, 50)
    model = node2vec_300(network, tw_dict)
    l = model.wv.vectors
    node = model.wv.key_to_index 
    generate_file(filename, output_name, l, node)

In [10]:
%%time
work_flow('D:/comp90049/project3/data/train_full.csv', 'train')
work_flow('D:/comp90049/project3/data/dev_full.csv', 'dev')
work_flow('D:/comp90049/project3/data/test_full.csv', 'test')

Computing transition probabilities:   0%|          | 0/3400 [00:00<?, ?it/s]

3400


Computing transition probabilities:   0%|          | 0/300 [00:00<?, ?it/s]

300


Computing transition probabilities:   0%|          | 0/300 [00:00<?, ?it/s]

300
Wall time: 3min 7s
